# 导入包

In [1]:
from collections import defaultdict
from tqdm import tqdm
import re
import json
import os
from copy import deepcopy
import random

# 读取大规模的KG

In [2]:
sub_map = defaultdict(list)
obj_map = defaultdict(list)
so_map=defaultdict(list)
kg_path='../knowledge/Knowledge.txt'
with open(kg_path) as f:
    lines=f.readlines()
    
entities_set=set()
relations_set=set()
print(len(lines))

#new_triples=[]
for i in tqdm(range(len(lines))):
    line=lines[i]
    l = line.strip().split('\t')
    s = l[0].strip()
    p = l[1].strip()
    o = l[2].strip()
    if s=='' or p=='' or o=='':
        print('-'*100)
        continue
    entities_set.add(s)
    entities_set.add(o)
    relations_set.add(p)
    #new_triples.append((s,p,o))
    sub_map[s].append((p, o))
    obj_map[o].append((s, p))
    so_map[(s,o)].append(p)

22883549


100%|███████████████████████████████████████████████████████████████████████████████████████| 22883549/22883549 [01:12<00:00, 314086.14it/s]


In [4]:
print('大规模图谱中一共有{}个三元组，{}个实体，{}个关系'.format(len(lines),len(entities_set),len(relations_set)))

大规模图谱中一共有22883549个三元组，9418104个实体，245837个关系


# 读取实体和关系

In [5]:
kg_folder='../knowledge/'
ent2id = {}
ent_path=os.path.join(kg_folder, 'entities.dict')
with open(ent_path) as f:
    lines=f.readlines()
for i in tqdm(range(len(lines))):
    l = lines[i].strip().split('\t')
    ent2id[l[0].strip()] = len(ent2id)
rel2id = {}
rel_path=os.path.join(kg_folder, 'relations.dict')
with open(rel_path) as f:
    lines=f.readlines()
for i in tqdm(range(len(lines))):
    l = lines[i].strip().split('\t')
    rel2id[l[0].strip()] = int(l[1])

100%|██████████████████████████████████████████████████████████████████████████████████████████| 245837/245837 [00:00<00:00, 1867843.73it/s]


In [6]:
print(len(ent2id),len(rel2id))

9418104 245837


# 读取提交的测试集

In [7]:
private_test_data=[]
with open("../qa_data/private_test.json") as f:
    lines=f.readlines()
    for line in lines:
        private_test_data.append(json.loads(line.strip()))

In [8]:
private_test_data[0]

{'id': 0, 'question': '我很好奇刘质平的老师是谁？', 'topic_entity': '刘质平'}

# 读取数据集的所有QA对

In [9]:
def read_qa(qa_file):
    data=[]
    with open(qa_file) as f:
        lines=f.readlines()
    for i in tqdm(range(len(lines))):
        line=lines[i]
        line = line.strip()
        line = line.split('\t')
        assert len(line)==2
        ans = line[1].strip()
        question=line[0]
        topic_entity=re.findall('<(.*)>',question)[0]
        data.append({"question":question,'topic_entity':topic_entity,'answer':ans})
    return data

In [10]:
data=[]
data.extend(read_qa('../qa_data/train.txt'))
data.extend(read_qa('../qa_data/test.txt'))

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2000/2000 [00:00<00:00, 913095.46it/s]


In [11]:
len(data)

21974

In [12]:
data[0]

{'question': '你知道<守望星光（韩玉玲、刘东考演唱歌曲）>是哪家公司出品的吗？',
 'topic_entity': '守望星光（韩玉玲、刘东考演唱歌曲）',
 'answer': '韩玉玲原创音乐'}

# 构造小规模的知识图谱

In [21]:
small_triplets=set()
small_entities=set()
small_relations=set()
bad_topic_entities=[]
bad_answers=[]
for i in tqdm(range(len(data))):
    example=deepcopy(data[i])
    question,topic_entity,answer=example['question'],example['topic_entity'],example['answer']
    assert topic_entity in ent2id and answer in ent2id
    if topic_entity not in sub_map:
        bad_topic_entities.append(topic_entity)
        continue
    elif answer not in obj_map:
        bad_answers.append(answer)
        continue
    else:
        head=topic_entity
        small_entities.add(head)
        small_entities.add(answer)
        for rel in so_map[(head,answer)]:
            small_relations.add(rel)
            
        for p1, o1 in sub_map[head]:
            if p1 in small_relations:
                small_triplets.add((head,p1,o1))
                small_entities.add(o1)
                #small_relations.add(p1)
#             for p2, o2 in sub_map[o1]:
#                 small_triplets.add((o1,p2,o2))
#                 small_entities.add(o2)
#                 small_relations.add(p2)

100%|██████████████████████████████████████████████████████████████████████████████████████████████| 21974/21974 [00:00<00:00, 98938.47it/s]


In [22]:
print(bad_answers,bad_topic_entities)

[] []


In [23]:
print(len(small_entities),len(small_triplets),len(small_relations))

130953 168797 2287


In [24]:
bad_topic_entities=[]

for i in tqdm(range(len(private_test_data))):
    example=deepcopy(private_test_data[i])
    question,topic_entity=example['question'],example['topic_entity']
    if topic_entity not in ent2id:
        bad_topic_entities.append(topic_entity)
        continue
    else:
        head=topic_entity
        small_entities.add(head)
            
        for p1, o1 in sub_map[head]:
            small_triplets.add((head,p1,o1))
            small_entities.add(o1)
            small_relations.add(p1)
#             for p2, o2 in sub_map[o1]:
#                 small_triplets.add((o1,p2,o2))
#                 small_entities.add(o2)
#                 small_relations.add(p2)

100%|███████████████████████████████████████████████████████████████████████████████████████████████| 3000/3000 [00:00<00:00, 136249.48it/s]


In [25]:
print(len(bad_topic_entities))

16


In [26]:
print(len(small_entities),len(small_triplets),len(small_relations))

146724 189599 4524


# 保存小规模的知识图谱

In [27]:
small_kg_folder='../knowledge/small_knowledge'
with open(os.path.join(small_kg_folder,"Knowledge.txt"),'w') as f:
    for head,rel,tail in small_triplets:
        f.write('\t'.join([head,rel,tail])+'\n')
        
with open(os.path.join(small_kg_folder,"entities.dict"),'w') as f:
    for i,ent in enumerate(small_entities):
        f.write(ent+'\t'+str(i)+'\n')
        
with open(os.path.join(small_kg_folder,"relations.dict"),'w') as f:
    for i,rel in enumerate(small_relations):
        f.write(rel+'\t'+str(i)+'\n')

373714